# 转换并量化中文Alpaca Plus模型

关于其他模型请参考另一个notebook：https://colab.research.google.com/drive/1Eak6azD3MLeb-YsfbP8UZC8wrL1ddIMI?usp=sharing


🎉🎉🎉 **新：现在免费用户也有机会能够转换7B和13B模型了！**

💡 提示和小窍门：
- 免费用户默认的内存只有12G左右，**笔者用免费账号实测选择TPU的话有机会随机出35G内存**，建议多试几次。如果能随机出25G内存以上的机器就可以了转换7B模型了，35G内存以上机器就能转换13B模型了
- Pro(+)用户请选择 “代码执行程序” -> “更改运行时类型” -> “高RAM”
- 实测：转换7B级别模型，25G内存的机器就够了；转换13B级别模型需要30G以上的内存（程序莫名崩掉或断开连接就说明内存爆了）
- 如果选了“高RAM”之后内存还是不够大的话，选择以下操作，有的时候会分配出很高内存的机器，祝你好运😄！
    - 可以把GPU或者TPU也选上（虽然不会用到）
    - 选GPU时，Pro用户可选“高级”类型GPU

以下信息配置信息供参考（Pro订阅下测试），运行时规格设置为“高RAM”时的设备配置如下（有随机性）：

| 硬件加速器  |  RAM  |  硬盘  |
| :-- | :--: | :--: |
| None | 25GB | 225GB |
| TPU | 35GB | 225GB |
| GPU（标准，T4）| 25GB | 166GB |
| GPU（高性能，V100）| 25GB | 166GB |
| GPU（高性能，A100）| **80GB** | 166GB |

*温馨提示：用完之后注意断开运行时，选择满足要求的最低配置即可，避免不必要的计算单元消耗（Pro只给100个计算单元）。*

## 安装相关依赖

In [ ]:
!pip install torch==1.12.0
!pip install transformers
!pip install git+https://github.com/huggingface/peft
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
peft 0.2.0 requires torch>=1.13.0, but you have torch 1.12.0 which is incompatible.
Looking in indexes: https:

## 克隆目录和代码

In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 761, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 761 (delta 54), reused 69 (delta 29), pack-reused 559
Receiving objects: 100% (761/761), 11.16 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (444/444), done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 2086, done.
remote: Counting objects: 100% (842/842), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 2086 (delta 778), reused 756 (delta 743), pack-reused 1244
Receiving objects: 100% (2086/2086), 2.12 MiB | 16.33 MiB/s, done.
Resolving deltas: 100% (1345/1345), done.


## 合并模型（Alpaca-Plus-7B）

**⚠️ 再次提醒：7B模型需要25G内存，13B模型需要35G+内存。**

此处使用的是🤗模型库中提供的基模型（已是HF格式），而不是Facebook官方的LLaMA模型，因此略去将原版LLaMA转换为HF格式的步骤。

**这里直接运行第二步：合并LoRA权重**，生成全量模型权重。可以直接指定🤗模型库的地址，也可以是本地存放地址。
- 基模型：`decapoda-research/llama-7b-hf` *（use at your own risk）*
- LoRA模型：先写`ziqingyang/chinese-llama-plus-lora-7b`然后再写`ziqingyang/chinese-alpaca-plus-lora-7b`
- 输出类型：因为后续要量化，这里将`output_type`设置为`pth`

💡 转换13B模型提示：
- 请将参数`--base_model`和`--lora_model`中的的`7b`改为`13b`即可
- **免费用户必须增加一个参数`--offload_dir`以缓解内存压力**，例如`--offload_dir ./offload_temp`

该过程比较耗时（下载+转换），需要几分钟到十几分钟不等，请耐心等待。
转换好的模型存放在`alpaca-combined`目录。
如果你不需要量化模型，那么到这一步就结束了。

In [ ]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora.py \
    --base_model decapoda-research/llama-7b-hf \
    --lora_model ziqingyang/chinese-llama-plus-lora-7b,ziqingyang/chinese-alpaca-plus-lora-7b \
    --output_type pth \
    --output_dir alpaca-combined

2023-04-28 08:07:00.276520: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Base model: decapoda-research/llama-7b-hf
LoRA model(s) ['ziqingyang/chinese-llama-plus-lora-7b', 'ziqingyang/chinese-alpaca-plus-lora-7b']:
Loading checkpoint shards: 100% 33/33 [01:18<00:00,  2.39s/it]
Peft version: 0.3.0.dev0
Loading LoRA for 7B model
Loading LoRA ziqingyang/chinese-llama-plus-lora-7b
Extended vocabulary size to 49953
Merging with merge_and_unload...
Loading LoRA ziqingyang/chinese-alpaca-plus-lora-7b
Extended vocabulary size to 49954
Merging with merge_and_unload...
Saving to pth format...
Saving shard 1 of 1 into alpaca-combined/consolidated.00.pth


## 量化模型
接下来我们使用[llama.cpp](https://github.com/ggerganov/llama.cpp)工具对上一步生成的全量版本权重进行转换，生成4-bit量化模型。

### 编译工具

首先对llama.cpp工具进行编译。

In [ ]:
!cd llama.cpp && make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

cc  -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -c llama.cpp -o llama.o
g++ -I. -I./examples -O3 -DNDEBUG -s

### 模型转换为ggml格式（FP16）

这一步，我们将模型转换为ggml格式（FP16）。
- 在这之前需要把`alpaca-combined`目录挪个位置，把模型文件放到`llama.cpp/zh-models/7B`下，把`tokenizer.model`放到`llama.cpp/zh-models`
- tokenizer在哪里？
    - `alpaca-combined`目录下有
    - 或者从以下网址下载：https://huggingface.co/ziqingyang/chinese-alpaca-lora-7b/resolve/main/tokenizer.model （注意，Alpaca和LLaMA的`tokenizer.model`不能混用！）

💡 转换13B模型提示：
- tokenizer可以直接用7B的，13B和7B的相同
- Alpaca和LLaMA的`tokenizer.model`不能混用！
- 以下看到7B字样的都是文件夹名，与转换过程没有关系了，改不改都行

In [ ]:
!cd llama.cpp && mkdir zh-models && mv ../alpaca-combined zh-models/7B
!mv llama.cpp/zh-models/7B/tokenizer.model llama.cpp/zh-models/
!ls llama.cpp/zh-models/

7B  tokenizer.model


In [ ]:
!cd llama.cpp && python convert.py zh-models/7B/

Loading model file zh-models/7B/consolidated.00.pth
Loading vocab file zh-models/tokenizer.model
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  49954 x   4096  | type UnquantizedDataType(name='F16')
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  49954 x   4096  | type UnquantizedDataType(name='F16')
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type UnquantizedDataType(name='F16')
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type UnquantizedDataType(name='F16')
[  6/291] Writing tensor layers.0.attention.wv.weight           | size   4096 x   4096  | type UnquantizedDataType(name='F16')
[  7/291] Writing tensor layers.0.attention.wo.weight           | size   4096 x   4096  | type UnquantizedDataType(name='F16

### 将FP16模型量化为8-bit

我们进一步将FP16模型转换为8-bit量化模型。

In [ ]:
!cd llama.cpp && ./quantize ./zh-models/7B/ggml-model-f16.bin ./zh-models/7B/ggml-model-q8_0.bin 7

llama.cpp: loading model from ./zh-models/7B/ggml-model-f16.bin
llama.cpp: saving model to ./zh-models/7B/ggml-model-q4_0.bin
[   1/ 291]                tok_embeddings.weight -     4096 x 49954, type =    f16, quantizing .. size =   390.27 MB ->   219.52 MB | hist: 0.000 0.027 0.020 0.032 0.048 0.067 0.088 0.106 0.225 0.106 0.088 0.067 0.048 0.032 0.020 0.027 
[   2/ 291]                          norm.weight -             4096, type =    f32, size =    0.016 MB
[   3/ 291]                        output.weight -     4096 x 49954, type =    f16, quantizing .. size =   390.27 MB ->   219.52 MB | hist: 0.000 0.027 0.020 0.032 0.048 0.067 0.088 0.106 0.226 0.106 0.088 0.067 0.048 0.032 0.020 0.027 
[   4/ 291]         layers.0.attention.wq.weight -     4096 x  4096, type =    f16, quantizing .. size =    32.00 MB ->    18.00 MB | hist: 0.000 0.026 0.018 0.028 0.044 0.064 0.088 0.111 0.245 0.111 0.087 0.064 0.044 0.028 0.018 0.026 
[   5/ 291]         layers.0.attention.wk.weight -     4096 

In [ ]:
!sha256sum ./llama.cpp/zh-models/7B/ggml-model-q8_0.bin

0eec8927427f159397c79961a28d62d78849514a4a19033b247edd6ac3fc2cfd  ./llama.cpp/zh-models/7B/ggml-model-q8_0.bin


### （可选）测试量化模型解码
至此已完成了所有转换步骤。
我们运行一条命令测试一下是否能够正常加载并进行对话。

FP16和Q8量化文件存放在./llama.cpp/zh-models/7B下，可按需下载使用。

In [ ]:
!cd llama.cpp && ./main -m ./zh-models/7B/ggml-model-q8_0.bin --color -f ./prompts/alpaca.txt -p "详细介绍一下北京的名胜古迹：" -n 512

main: seed = 1682671021
llama.cpp: loading model from ./zh-models/7B/ggml-model-q8_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 9180.12 MB (+ 1026.00 MB per state)
llama_init_from_file: kv self size  =  256.00 MB

system_info: n_threads = 4 / 4 | AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0